In [ ]:
from bs4 import BeautifulSoup 
from csv import writer

In [ ]:
import requests

# Kullanıcı aracısı dizesini istediğiniz gibi değiştirin
custom_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

# İstek başlıklarını ayarlayarak kullanıcı aracısı dizesini değiştirin
headers = {
    "User-Agent": custom_user_agent
}

# İstek gönderme
response = requests.get("https://turbo.az/", headers=headers)

# Yanıtı kontrol etme
if response.status_code == 200:
    print("confirmed!")
else:
    print("not confirmed")


In [ ]:
import pandas as pd 
import numpy as np 
import requests 
from bs4 import BeautifulSoup
import threading
import time
from datetime import datetime


In [ ]:
print('started')

In [ ]:
exc_start = time.time()
page_count = 23
urls = list()

In [ ]:
for page in range(1, page_count+1):
    url = f'https://turbo.az/autos?page={page}'
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    items = soup.find_all('a', class_='products-i__link')
    urls.append([a_tag.get('href') for a_tag in items]) 

In [ ]:
urls = np.concatenate(urls, axis=None)
urls = np.array(['https://turbo.az' + url for url in urls])
urls = np.unique(urls)

print(f'Url Handling: {time.time() - exc_start}')

In [ ]:
data_list = list()
errors = list()


In [ ]:
def scrape(url):
    
    """
    Scrapes data from a webpage based on the provided URL.

    Args:
        url (str): The URL of the webpage to scrape.
    """
    
    data = pd.DataFrame()
    
    
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    
    owner = soup.find('div', 'product-owner__info')
        
    try:
        data['Telefonlar'] = ' && '.join([i.text.strip() for i in soup.find_all('a', class_='product-phones__list-i')])
    except: errors.append(('Telefon', url))
    
   
    data['Url'] = [url]
    
    data_list.append(data)

In [ ]:
url_count = len(urls)
thread_count = 35

In [ ]:
arr = [*range(0, url_count, thread_count)]
length = [*range(url_count)]
i = 0

In [ ]:
while True:
    all_threads = []
    try:
        for e in length[arr[i]:arr[i + 1]]:
            thread = threading.Thread(target=scrape, args=(urls[e],))
            all_threads.append(thread)

        for thread in all_threads:
            thread.start()

        for thread in all_threads:
            thread.join()

        i += 1
    except:
        for e in length[arr[i]:]:
            thread = threading.Thread(target=scrape, args=(urls[e],))
            all_threads.append(thread)

        for thread in all_threads:
            thread.start()

        for thread in all_threads:
            thread.join()
        break  

In [ ]:
main_data = pd.concat(data_list, ignore_index=True)
main_data.to_csv(f'{datetime.now().strftime("%b-%d-%Y %H-%M-%S")}.csv',
                 encoding='utf_8_sig',
                 index=False)


In [ ]:
print(*errors, end='\n')
print(f'Execution time: {time.time() - exc_start}')

In [ ]:
main_data